In [15]:

from Pylette import extract_colors
from matplotlib import pyplot as plt
import numpy as np
from pyemd import emd
import os

In [16]:



def extract_palette_rgb(image_path):
    colors = extract_colors(image = image_path, palette_size=10, mode="KMeans")
    rgb_array = np.array([color.rgb for color in colors], dtype=np.float64)  # shape Nx3
    freq_array = np.array([color.freq for color in colors], dtype=np.float64)  # shape N
    freq_array /= freq_array.sum()

    return rgb_array, freq_array

from matplotlib import pyplot as plt
import numpy as np

def visualize_palette(rgb_array, freq_array):
    # Normalize RGB to [0, 1]
    rgb_array = rgb_array / 255.0

    # Create an image strip using frequency as width proportions
    total_width = 800
    height = 100
    img = np.zeros((height, total_width, 3))

    start = 0
    for color, freq in zip(rgb_array, freq_array):
        width = int(freq * total_width)
        end = start + width
        img[:, start:end, :] = color
        start = end

    plt.figure(figsize=(8, 2))
    plt.imshow(img)
    plt.axis("off")
    plt.show()



In [17]:
print(extract_palette_rgb('train_data/dark_forest/2036_2035_6102.png'))


(array([[142., 180., 254.],
       [ 18.,  33.,   9.],
       [ 26.,  52.,  14.],
       [ 37.,  71.,  20.],
       [  5.,  11.,   3.],
       [ 46.,  90.,  25.],
       [165., 197., 254.],
       [ 45.,  33.,  19.],
       [ 58., 113.,  33.],
       [114., 137., 153.]]), array([0.25254822, 0.17074585, 0.16218567, 0.11950684, 0.09263611,
       0.07095337, 0.05955505, 0.03872681, 0.03053284, 0.00260925]))


In [18]:
# ----------------------------
# Step 2: Compute EMD distance
# ----------------------------
def palette_emd(palette1, palette2, frequency1, frequency2):
    """
    Compute order-independent EMD between two RGB palettes.
    Assumes both palettes have same number of colors (10 here).
    """
    n = len(palette1)
    # Distance matrix (Euclidean in RGB)
    dist_matrix = np.linalg.norm(palette1[:, None, :] - palette2[None, :, :], axis=2).astype(np.float64)
    return emd(frequency1, frequency2, dist_matrix)

# ----------------------------
# Step 3: KNN classifier
# ----------------------------
class EMDBasedKNN:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
        self.palettes = []
        self.labels = []

    def fit(self, palettes, labels,):
        self.palettes = palettes
        self.labels = labels

    def predict(self, new_palette):
        distances = [palette_emd(new_palette[0], p[0], new_palette[1], p[1]) for p in self.palettes]
        sorted_indices = np.argsort(distances)[:self.n_neighbors]
        neighbor_labels = [self.labels[i] for i in sorted_indices]
        # Majority vote
        return max(set(neighbor_labels), key=neighbor_labels.count)

In [19]:
def load_training_data(train_folder):
    palettes = []
    labels = []
    for biome_name in os.listdir(train_folder):
        biome_path = os.path.join(train_folder, biome_name)
        if not os.path.isdir(biome_path):
            continue
        for img_file in os.listdir(biome_path):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(biome_path, img_file)
                palette = extract_palette_rgb(img_path)
                palettes.append(palette)
                labels.append(biome_name)
    return palettes, labels

In [20]:
X, y = load_training_data('train_data')

KNN = EMDBasedKNN(n_neighbors=3)
KNN.fit(X, y)



/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/lib/python3.13/site-p

In [21]:

KNN.predict(extract_palette_rgb('eval_data/2014_4182_4713.png'))


ValueError: Histogram lengths must be equal